## Homework

The goal of this homework is to get familiar with tools like MLflow for experiment tracking and 
model management.




## Q1. Install the package

To get started with MLflow you'll need to install the appropriate Python package.

For this we recommend creating a separate Python environment, for example, you can use [conda environments](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-envs), 
and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

What's the version that you have?


In [1]:
!mlflow --version

mlflow, version 2.3.2


## Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset to predict the amount of tips for each trip. 

Download the data for January, February and March 2022 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](homework) to preprocess the data.

The script will:

* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2022 data),
* save the preprocessed datasets and the `DictVectorizer` to disk.

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

So what's the size of the saved `DictVectorizer` file?

* 54 kB
* 154 kB
* 54 MB
* 154 MB


In [2]:
! wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet -P ./data/
! wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-02.parquet -P ./data/
! wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-03.parquet -P ./data/

--2023-05-26 11:49:04--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:200c:6200:b:20a5:b140:21, 2600:9000:200c:8c00:b:20a5:b140:21, 2600:9000:200c:1e00:b:20a5:b140:21, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:200c:6200:b:20a5:b140:21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1254291 (1,2M) [binary/octet-stream]
Saving to: './data/green_tripdata_2022-01.parquet'

     0K .......... .......... .......... .......... ..........  4%  300K 4s
    50K .......... .......... .......... .......... ..........  8%  534K 3s
   100K .......... .......... .......... .......... .......... 12% 1,07M 2s
   150K .......... .......... .......... .......... .......... 16%  692K 2s
   200K .......... .......... .......... .......... .......... 20% 2,25M 2s
   250K .......... .......... .......... .......... ...

In [6]:
!python ./preprocess_data.py --raw_data_path ./data --dest_path ./output

In [12]:
import os
print(f'DictVectorizer size: {round(os.path.getsize("./output/dv.pkl")/1024,0)} kB')

DictVectorizer size: 150.0 kB




## Q3. Train a model with autolog

We will train a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder `homework`. 

The script will:

* load the datasets produced by the previous step,
* train the model on the training set,
* calculate the RMSE score on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

What is the value of the `max_depth` parameter:

* 4
* 6
* 8
* 10


In [14]:
!python ./train.py

In [17]:
print('max_depth: 10')

max_depth: 10




## Launch the tracking server locally for MLflow

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

In case of MLflow, you need to:

* launch the tracking server on your local machine,
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

You should keep the tracking server running to work on the next three exercises that use the server.


## Q4. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `optuna`. 
We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization,
* the RMSE obtained on the validation set (February 2022 data).

What's the best validation RMSE that you got?

* 1.85
* 2.15
* 2.45
* 2.85


In [18]:
!pip install optuna

     -------------------------------------- 365.7/365.7 kB 5.7 MB/s eta 0:00:00


In [19]:
!python ./hpo.py

2023/05/26 12:27:20 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
[I 2023-05-26 12:27:20,941] A new study created in memory with name: no-name-3b51589b-c67a-4fa4-b11a-256316986f35
[I 2023-05-26 12:27:24,019] Trial 0 finished with value: 2.451379690825458 and parameters: {'n_estimators': 25, 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 3}. Best is trial 0 with value: 2.451379690825458.
[I 2023-05-26 12:27:24,385] Trial 1 finished with value: 2.4667366020368333 and parameters: {'n_estimators': 16, 'max_depth': 4, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 0 with value: 2.451379690825458.
[I 2023-05-26 12:27:26,919] Trial 2 finished with value: 2.449827329704216 and parameters: {'n_estimators': 34, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 2 with value: 2.449827329704216.
[I 2023-05-26 12:27:27,901] Trial 3 finished with value: 2.460983516558473 a

In [38]:
import mlflow
from mlflow.entities import ViewType

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("random-forest-hyperopt")

experiments = mlflow.search_experiments(view_type=ViewType.ALL)
for exp in experiments:
    print(exp.name)

random-forest-hyperopt
homework-02-experiment-tracking
Default


In [74]:
import mlflow 
client = mlflow.tracking.MlflowClient()
runs = client.search_runs(experiment_ids = "2", order_by=["metrics.rmse ASC"])
best_run = runs[0]
print(f'best validation rmse: {round(best_run.data.metrics.get("rmse"),2)}')

best validation rmse: 2.45


## Q5. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. 
In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. 
After that, it will calculate the RMSE of those models on the test set (March 2022 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tips for MLflow:

* you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE,
* to register the model you can use the method `mlflow.register_model` and you will need to pass the right `model_uri` in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

* 1.885
* 2.185
* 2.555
* 2.955


In [65]:
!python register_model.py

Registered model 'best_model' already exists. Creating a new version of this model...
2023/05/26 14:12:16 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: best_model, version 1
Created version '1' of model 'best_model'.


In [73]:
EXPERIMENT_NAME = "random-forest-best-models"

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(EXPERIMENT_NAME)

experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
best_run = client.search_runs(experiment.experiment_id, order_by=["metrics.test_rmse ASC"])[0]
print(f'test RMSE of the best model: {round(best_run.data.metrics.get("test_rmse"),3)}')


test RMSE of the best model: 2.285


## Q6. Model metadata

Now explore your best model in the model registry using UI. What information does the model registry contain about each model?

* Version number
* Source experiment
* Model signature
* All the above answers are correct


answer:
- version number
- source run
- model signature



## Submit the results

* Submit your results here: https://forms.gle/Fy1pvrPEKd4yjz3s6
* You can submit your solution multiple times. In this case, only the last submission will be used
* If your answer doesn't match options exactly, select the closest one


## Deadline

The deadline for submitting is 1 June 2023 (Thursday), 23:00 CEST (Berlin time). 

After that, the form will be closed.
